In [1]:
#@title Leitura das bibliotecas
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
##Remove qualquer outro modelo existente do H20 Python
!pip uninstall h2o
#
##instalando o módulo do H20 Python
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o
#
##Verificando a versão mais recente
!python -m pip install --upgrade pip


Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 128.4MB 74kB/s 
  Created wheel for h2o: filename=h2o-3.30.0.6-py2.py3-none-any.whl size=128412300 sha256=42c0365b61f07f14d511806cc596dc2e584026ba0287f9d100061f0859453fe1
  Stored in directory: /root/.cache/pip/wheels/de/4c/dd/4813e95c4b5328b4de2e2e1aa56ca66a547f45d11fe47c3c8a
Successfully built h2o
     |████████████████████████████████| 1.5MB 2.6MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [5]:
#@title Import biblioeca H2O
import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpgc7tm8zl
  JVM stdout: /tmp/tmpgc7tm8zl/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpgc7tm8zl/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.6
H2O_cluster_version_age:,1 day
H2O_cluster_name:,H2O_from_python_unknownUser_hmx6jl
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [7]:
#@title Importando a base de treino e teste
train = h2o.import_file("/content/V2_train_df.csv")
test = h2o.import_file("/content/V2_test_df.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
#@title Exclusao de variáveis futuras
train = train.drop(['evolucao_Infarto após Intervenção',
'evolucao_Óbito Hospitalar',
'evolucao_Complicação Vascular',
'evolucao_I.R.A',
'evolucao_I.R.A Dialítica',
'evolucao_A.V.C Hemorrágico',
'evolucao_A.V.C Isquêmico'
'paciente_Nº da Intervenção',
'procedimento_Nº. Paciente',
'procedimento_Nº da Intervenção',
'procedimento_Nº. Paciente',
'procedimento_Nº da Intervenção',
'complicacao_Nº. Paciente',
'complicacao_Nº da Intervenção',
'balao_Nº. Paciente',
'balao_Nº da Intervenção',
'stent_Nº. Paciente',
'stent_Nº da Intervenção',
'vaso_Nº. Paciente,'
'vaso_Nº da Intervenção')
test=test.drop(['evolucao_Infarto após Intervenção',
'evolucao_Óbito Hospitalar',
'evolucao_Complicação Vascular',
'evolucao_I.R.A',
'evolucao_I.R.A Dialítica',
'evolucao_A.V.C Hemorrágico',
'evolucao_A.V.C Isquêmico'], axis=1)

In [ ]:
#@title Identificando variáveis explicativas e variável resposta
x = train.columns
y = "evolucao_Problema Clínico"
x.remove(y)

In [ ]:
#@title Definindo a variável resposta como um fator
test[y] = test[y].asfactor()

In [ ]:
#@title Gerando modelo de AutoML limitando o tempo de treinamento a 300 segundos
aml = H2OAutoML(max_runtime_secs = 300)
aml.train(x = x, y = y,
          training_frame = train,
          leaderboard_frame = test)

AutoML progress: |████████████████████
03:14:25.915: XRT_1_AutoML_20200630_031238 [DRF XRT (Extremely Randomized Trees)] failed: java.lang.AssertionError

███████████████████████████████████| 100%


In [ ]:
#@title Visualização do melhor modelo
lb = aml.leaderboard
lb

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
XGBoost_grid__1_AutoML_20200630_031238_model_2,0.913975,0.10494,0.434845,0.268161,0.166374,0.0276802
StackedEnsemble_BestOfFamily_AutoML_20200630_031238,0.911835,0.110073,0.497104,0.21918,0.165035,0.0272364
StackedEnsemble_AllModels_AutoML_20200630_031238,0.909115,0.107762,0.502968,0.240678,0.163724,0.0268055
XGBoost_1_AutoML_20200630_031238,0.907116,0.113202,0.478822,0.245985,0.162286,0.0263368
XGBoost_grid__1_AutoML_20200630_031238_model_1,0.906123,0.11365,0.445715,0.254828,0.166508,0.027725
XGBoost_3_AutoML_20200630_031238,0.902358,0.106866,0.465311,0.228705,0.16201,0.0262472
XGBoost_2_AutoML_20200630_031238,0.900196,0.10786,0.474034,0.223943,0.1639,0.0268632
XGBoost_grid__1_AutoML_20200630_031238_model_3,0.899416,0.10725,0.445267,0.229521,0.166079,0.0275822
XGBoost_grid__1_AutoML_20200630_031238_model_4,0.898524,0.11215,0.471763,0.223671,0.163687,0.0267933
GBM_grid__1_AutoML_20200630_031238_model_1,0.894258,0.108624,0.439003,0.224215,0.166042,0.0275699


In [ ]:
#@title Melhor modelo
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_grid__1_AutoML_20200630_031238_model_2


Model Summary: 


,,number_of_trees
0,,41.0




ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.020299135513959274
RMSE: 0.14247503470418693
LogLoss: 0.07846859521351014
Mean Per-Class Error: 0.09423474251207509
AUC: 0.963132013397004
AUCPR: 0.7227598013165976
Gini: 0.9262640267940081

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.28280330697695416: 


,,0,1,Error,Rate
0,0,5592.0,54.0,0.0096,(54.0/5646.0)
1,1,90.0,143.0,0.3863,(90.0/233.0)
2,Total,5682.0,197.0,0.0245,(144.0/5879.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-06-30 03:15:14,48.090 sec,0.0,0.500000,0.693147,0.500000,0.039633,1.000000,0.960367
1,,2020-06-30 03:15:15,48.727 sec,5.0,0.195703,0.184472,0.931984,0.391908,14.687144,0.053410
2,,2020-06-30 03:15:16,49.406 sec,10.0,0.169052,0.118709,0.933481,0.460686,17.533935,0.039462
3,,2020-06-30 03:15:16,49.704 sec,15.0,0.162698,0.102893,0.936684,0.518562,19.244562,0.032659
4,,2020-06-30 03:15:16,49.989 sec,20.0,0.156913,0.094860,0.944142,0.587468,21.810504,0.034360
5,,2020-06-30 03:15:16,50.258 sec,25.0,0.153001,0.089892,0.950925,0.632432,23.093475,0.029087
6,,2020-06-30 03:15:17,50.522 sec,30.0,0.149424,0.085670,0.955570,0.662488,23.521132,0.030958
7,,2020-06-30 03:15:17,50.868 sec,35.0,0.145846,0.082029,0.958189,0.693050,23.948789,0.026025
8,,2020-06-30 03:15:17,51.169 sec,40.0,0.142882,0.078998,0.962440,0.717108,24.376446,0.025004
9,,2020-06-30 03:15:17,51.330 sec,41.0,0.142475,0.078469,0.963132,0.722760,24.376446,0.024494



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,complicacao_Grau-Complicação_Grave_sum,218.177307,1.000000,0.124102
1,stent_Timipós_mean,139.857132,0.641025,0.079552
2,balao_Graudeestenosepós_mean,88.991211,0.407885,0.050619
3,balao_Graudeestenosepós_max,74.249023,0.340315,0.042234
4,paciente_ÍndicedeMassaCorporal,65.166855,0.298688,0.037068
5,balao_ResultadoAngiográfico-Balão_S-Sucesso_sum,60.797695,0.278662,0.034582
6,evolucao_NºdaIntervenção,60.674564,0.278098,0.034512
7,balao_Graudeestenosepós_min,56.596832,0.259408,0.032193
8,procedimento_Qtde(ml)_max,48.458961,0.222108,0.027564
9,procedimento_DuraçãoInterv.(min)_max,44.250458,0.202819,0.025170



See the whole table with table.as_data_frame()


In [ ]:
#@title Predição
preds = aml.leader.predict(test)
preds

xgboost prediction progress: |████████████████████████████████████████████| 100%


predict,p0,p1
0,0.988345,0.0116552
0,0.940629,0.0593706
0,0.997775,0.002225
0,0.986522,0.013478
0,0.993869,0.006131
0,0.995631,0.00436937
0,0.995698,0.00430237
0,0.994273,0.0057268
0,0.995964,0.00403559
0,0.98849,0.0115096


In [4]:
#@title Predição
preds = aml.leader.predict(test)
preds

NameError: ignored